In [1]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from copy import deepcopy
from datetime import datetime
from tqdm import tqdm
from functools import reduce
import unidecode
import re
import operator

import plotly 
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.offline as offline
offline.init_notebook_mode(connected=True)

from IPython.display import clear_output
import requests

# con = create_engine(server[db_cts])
plotly.__version__
%matplotlib inline

## Basic informations table

### Importing data

In [2]:
deputados_detalhes = pd.read_sql_query("""
    SELECT DISTINCT
      t1.idecadastro,
      numlegislatura,
      nomeparlamentaratual,
      nomecivil,
      datanascimento,
      partidoatualsigla,
      ufrepresentacaoatual,
      situacaonalegislaturaatual
    FROM
      c_camdep.camdep_deputados_detalhes t1,
      (SELECT DISTINCT
          idecadastro,
          max(capnum) OVER (PARTITION BY idecadastro) capnum
       FROM
          c_camdep.camdep_deputados_detalhes) t2
    WHERE
      t1.idecadastro= t2.idecadastro AND
      t1.capnum = t2.capnum AND
      numlegislatura in (52,53,54,55);
""",con)
print('deputados_datalhes OK')

tse_total_votos = pd.read_csv('data/total_votos_2014.csv')
print('tse_total_votos OK')

tse_candidatos = pd.read_sql("""
    SELECT
      sigla_uf,
      nome_candidato nomecivil,
      numero_candidato numero_votavel,
      data_nascimento datanascimento
    FROM
      c_tse.candidatos_2014
    WHERE
      codigo_cargo = 6 AND
      desc_sit_tot_turno IN ('ELEITO','ELEITO POR MÉDIA','ELEITO POR QP','SUPLENTE')
""",con)
print('tse_candidatos OK')

deputados_datalhes OK
tse_total_votos OK
tse_candidatos OK


In [ ]:
deputados_detalhes.count()

### Count legislatures

In [3]:
legs = deputados_detalhes[['idecadastro','numlegislatura']]\
       .groupby('idecadastro').count().to_dict()['numlegislatura']

dep_leg55 = deepcopy(deputados_detalhes.loc[deputados_detalhes.numlegislatura == 55])
dep_leg55['legislaturas'] = dep_leg55.idecadastro.map(legs)

### Get URL Picture

In [4]:
def get_urlfoto(row):
    return 'http://www.camara.leg.br/internet/deputado/bandep/{}.jpg'.format(row.idecadastro)

dep_leg55.loc[:,'urlfoto'] = dep_leg55.apply(get_urlfoto,axis=1)
dep_leg55.head(2)

,idecadastro,numlegislatura,nomeparlamentaratual,nomecivil,datanascimento,partidoatualsigla,ufrepresentacaoatual,situacaonalegislaturaatual,legislaturas,urlfoto
9,4930,55,SILVIO COSTA,SÍLVIO SERAFIM COSTA,1956-12-23,AVANTE,PE,Em Exercício,3,http://www.camara.leg.br/internet/deputado/ban...
12,4931,55,IZALCI LUCAS,IZALCI LUCAS FERREIRA,1956-04-07,PSDB,DF,Em Exercício,3,http://www.camara.leg.br/internet/deputado/ban...


### Votes processing

In [5]:
tse_total_votos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7041 entries, 0 to 7040
Data columns (total 5 columns):
numero_votavel    7040 non-null float64
sigla_uf          7041 non-null object
nome_partido      6986 non-null object
nome_votavel      7040 non-null object
total_votos       7041 non-null int64
dtypes: float64(1), int64(1), object(3)
memory usage: 275.1+ KB


In [6]:
tse_total_votos = tse_total_votos.dropna()
tse_total_votos.numero_votavel = tse_total_votos.numero_votavel.astype(int)

votes_by_parties = tse_total_votos[['nome_partido','total_votos']].groupby('nome_partido').sum()

def percent_party(row):
    return row.total_votos/votes_by_parties.total_votos[row.nome_partido]

tse_total_votos.loc[:,'percent_party'] = tse_total_votos.apply(percent_party,axis=1)

### Merging the TSE and CAMDEP congressmen datasets

In [7]:
tse_candidatos.numero_votavel = tse_candidatos.numero_votavel.astype(int)

dep_leg55.datanascimento = pd.to_datetime(dep_leg55.datanascimento).dt.date
tse_candidatos.datanascimento = pd.to_datetime(tse_candidatos.datanascimento).dt.date

In [8]:
def tse_unique_id(row):
    return '{}_{}'.format(row.sigla_uf,row.numero_votavel)

tse_candidatos.loc[:,'tse_id'] = tse_candidatos.apply(tse_unique_id,axis=1)
tse_total_votos.loc[:,'tse_id'] = tse_total_votos.apply(tse_unique_id,axis=1)

tse_candidatos = tse_candidatos.merge(
    tse_total_votos[['total_votos','percent_party','tse_id']],
    left_on='tse_id',right_on='tse_id',how='left'
)

In [9]:
def global_unique_id(row):
    name = row.nomecivil.upper().replace(' DE ','').replace(' ','')
    name = unidecode.unidecode(name)
    birth = row.datanascimento.year
    return '{}{}'.format(birth,name)

tse_candidatos.loc[:,'global_id'] = tse_candidatos.apply(global_unique_id,axis=1)
dep_leg55.loc[:,'global_id'] = dep_leg55.apply(global_unique_id,axis=1)

### Fixing noise

corrects_id = {
    74558:'1957GIVALDOSAGOUVEIACARIMBAO',
    146949:'1973ROSEANECAVALCANTEFREITAS',
    178851:'1979JOZIANEARAUJONASCIMENTOROCHA',
    178882:'1989ANDRELUISCARVALHORIBEIRO',
    178883:'1977ELIZIANEPEREIRAGAMAFERREIRA',
    178999:'1989BRUNIELEFERREIRADASILVA'
}

for i in corrects_id: 
    dep_leg55.loc[dep_leg55.idecadastro == i,'global_id'] = corrects_id[i]
    
dep_leg55 = dep_leg55.merge(
   tse_candidatos[['global_id','nomecivil','total_votos','percent_party','numero_votavel']],
   left_on='global_id',right_on='global_id',how='left'
)

dep_leg55.columns

Index(['idecadastro', 'numlegislatura', 'nomeparlamentaratual', 'nomecivil_x',
       'datanascimento', 'partidoatualsigla', 'ufrepresentacaoatual',
       'situacaonalegislaturaatual', 'legislaturas', 'urlfoto', 'global_id',
       'nomecivil_y', 'total_votos', 'percent_party', 'numero_votavel'],
      dtype='object')

In [10]:
dep_leg55 = dep_leg55.loc[:,[
    'global_id','idecadastro','numero_votavel','nomeparlamentaratual',
    'nomecivil_y','datanascimento','ufrepresentacaoatual','partidoatualsigla',
    'situacaonalegislaturaatual','urlfoto','total_votos','percent_party', 'legislaturas'
]]

dep_leg55.columns = [
    'global_id','idecadastro','numero_eleicao','nome_social',
    'nome_civil','data_nascimento','uf_eleicao','partido_atual',
    'situacao_exercicio','url_foto','total_votos','percent_party', 'mandatos_desde2003'
]
dep_leg55.head(2)

,global_id,idecadastro,numero_eleicao,nome_social,nome_civil,data_nascimento,uf_eleicao,partido_atual,situacao_exercicio,url_foto,total_votos,percent_party,mandatos_desde2003
0,1956SILVIOSERAFIMCOSTA,4930,2020.0,SILVIO COSTA,SILVIO SERAFIM COSTA,1956-12-23,PE,AVANTE,Em Exercício,http://www.camara.leg.br/internet/deputado/ban...,103461.0,0.040737,3
1,1956IZALCILUCASFERREIRA,4931,4545.0,IZALCI LUCAS,IZALCI LUCAS FERREIRA,1956-04-07,DF,PSDB,Em Exercício,http://www.camara.leg.br/internet/deputado/ban...,71937.0,0.006480,3


## Propositions table

### Importing Data

In [11]:
proposicoes_id = pd.read_sql("""
    SELECT DISTINCT
      t1.idproposicao,
      nomeproposicao,
      tema,
      ementa,
      idecadastro,
      autor,
      ufautor,
      partidoautor,
      dataapresentacao,
      regimetramitacao,
      indexacao,
      linkinteiroteor,
      situacao
    FROM
      c_camdep.camdep_proposicoes_id t1,
      (SELECT DISTINCT
         idproposicao,
         max(capnum) OVER (PARTITION BY idproposicao) capnum
       FROM
         c_camdep.camdep_proposicoes_id) t2
    WHERE
      t1.idproposicao = t2.idproposicao AND
      t1.capnum = t2.capnum AND
      t1.nomeproposicao SIMILAR TO '(PL|PLP|PLV|MPV|PDC|PEC) %%'
""",con)
print('proposicoes_id OK')

proposicoes_id OK


### Cataloging propositions by situation

Based on [analisys](https://docs.google.com/document/d/1qqcfJMybpvjKJ63i77YcVbIUzUezkTpnnqbBSJmXMT0/edit?usp=sharing) produced by Congresso em Números.

In [12]:
proposicoes_id.rename(columns={'situacao':'descricao_situacao'},inplace=True)
proposicoes_id['situacao'] = 'Em tramitação'

classification = {
    'Aprovada': [
        'Tranformada',
        'Transformado em Norma Jurídica',    
        'Aguardando Apreciação pelo Senado Federal',
        'Aguardando Envio ao Senado Federal',
        'Enviada ao Senado Federal',
        'Aguardando Promulgação',
        'Aguardando Apreciação do Veto',
        'Aguardando Sanção',
        'Aguardando Envio ao Executivo',
        'Vetado Totalmente',
        'Originou'    
    ],
    'Arquivada': [
        'Arquivada',
        'Aguardando Despacho de Arquivamento',
        'Aguardando Remessa ao Arquivo',
        'Enviada ao arquivo',
        'Perdeu a eficácia',
        'Transformado em nova proposição'
    ]
}

for situation in tqdm(classification):
    sits = '|'.join(classification[situation]) 
    proposicoes_id.loc[proposicoes_id['descricao_situacao'].str.contains(sits) == True, 'situacao'] = situation

100%|██████████| 2/2 [00:00<00:00,  2.62it/s]


### Count sponsors

In [13]:
proposicoes_id.idecadastro = np.nan_to_num(proposicoes_id.idecadastro).astype(int)
sponsors = pd.DataFrame(proposicoes_id.idecadastro.value_counts())
sponsors_approved = pd.DataFrame(
    proposicoes_id[proposicoes_id.situacao == 'Aprovada'].idecadastro.value_counts()
)
sponsors.columns = ['apresentacoes']
sponsors_approved.columns = ['apresentacoes_aprovadas']

dep_leg55 = dep_leg55\
    .merge(sponsors, left_on='idecadastro', right_index=True, how='left')\
    .merge(sponsors_approved, left_on='idecadastro', right_index=True, how='left')

dep_leg55.apresentacoes = np.nan_to_num(dep_leg55.apresentacoes).astype(int)
dep_leg55.apresentacoes_aprovadas = np.nan_to_num(dep_leg55.apresentacoes_aprovadas).astype(int)

dep_leg55.loc[:,'taxa_sucesso'] = dep_leg55.apresentacoes_aprovadas/dep_leg55.apresentacoes
dep_leg55.loc[:,'media_apresentacoes'] = np.average(dep_leg55.apresentacoes)
dep_leg55.loc[:,'media_taxa_sucesso'] = dep_leg55.taxa_sucesso.sum()/len(dep_leg55.taxa_sucesso)

dep_leg55.head(2)

,global_id,idecadastro,numero_eleicao,nome_social,nome_civil,data_nascimento,uf_eleicao,partido_atual,situacao_exercicio,url_foto,total_votos,percent_party,mandatos_desde2003,apresentacoes,apresentacoes_aprovadas,taxa_sucesso,media_apresentacoes,media_taxa_sucesso
0,1956SILVIOSERAFIMCOSTA,4930,2020.0,SILVIO COSTA,SILVIO SERAFIM COSTA,1956-12-23,PE,AVANTE,Em Exercício,http://www.camara.leg.br/internet/deputado/ban...,103461.0,0.040737,3,22,0,0.0,41.804241,0.037442
1,1956IZALCILUCASFERREIRA,4931,4545.0,IZALCI LUCAS,IZALCI LUCAS FERREIRA,1956-04-07,DF,PSDB,Em Exercício,http://www.camara.leg.br/internet/deputado/ban...,71937.0,0.006480,3,43,0,0.0,41.804241,0.037442


In [14]:
proposicoes_id.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133159 entries, 0 to 133158
Data columns (total 14 columns):
idproposicao          133159 non-null int64
nomeproposicao        133159 non-null object
tema                  62888 non-null object
ementa                133157 non-null object
idecadastro           133159 non-null int64
autor                 133159 non-null object
ufautor               107898 non-null object
partidoautor          107770 non-null object
dataapresentacao      133159 non-null datetime64[ns]
regimetramitacao      133064 non-null object
indexacao             132319 non-null object
linkinteiroteor       65285 non-null object
descricao_situacao    133138 non-null object
situacao              133159 non-null object
dtypes: datetime64[ns](1), int64(2), object(11)
memory usage: 14.2+ MB


## Themes by congresmen

### Get themes

In [15]:
themes = set()
for i in proposicoes_id.tema:
    if i:
        i = i.replace(' (utilizado até a legislatura 53)','')\
             .replace('; ',', ')\
             .split(', ')
        themes = themes.union(i)

# Filter propositions 

proposicoes_temas = proposicoes_id[['idecadastro','tema']][
    proposicoes_id.idecadastro.isin(dep_leg55.idecadastro)
]

In [16]:
proposicoes_temas.head(2)

,idecadastro,tema
12,73666,Organização Político-Administrativa do Estado ...
25,160515,Homenagens e Datas Comemorativas


### Count themes by congressman

In [17]:
parlamentares_tema = pd.DataFrame(columns=['idecadastro'] + list(themes))

dic = {theme:0 for theme in themes}
for i in dep_leg55.idecadastro.unique():
    dic['idecadastro'] = i
    parlamentares_tema = parlamentares_tema.append(dic,ignore_index=True)
parlamentares_tema = parlamentares_tema.astype(int)

In [18]:
k = 0
for l in tqdm(proposicoes_temas.to_dict(orient='split')['data']):
    k+=1
    if k%50 == 0: clear_output()
    if l[1]:
        l[1] = l[1].replace(' (utilizado até a legislatura 53)','')\
                   .replace('; ',', ')\
                   .split(', ')
        for theme in l[1]:
            parlamentares_tema.loc[parlamentares_tema.idecadastro == l[0],theme] += 1
parlamentares_tema.head()

100%|██████████| 25626/25626 [02:32<00:00, 168.26it/s]


,idecadastro,Desporto e Lazer,Ciência,Organização Administrativa do Estado,Homenagens e Datas Comemorativas,Processo Legislativo,Comércio Exterior,Direitos Humanos e Minorias,Partidária e Eleitoral,Economia,...,Cidades e Desenvolvimento Urbano,Viação e Transportes,Direito Civil e Processual Civil,Direito e Justiça,Recursos Hídricos,Direitos Humanos,Relações Internacionais,Política Fundiária,Finanças Públicas e Orçamento,Tributação
0,4930,0,0,0,0,0,0,0,2,2,...,0,1,1,0,1,2,0,0,1,0
1,4931,1,1,0,1,0,0,0,3,0,...,0,2,0,0,0,6,0,0,2,8
2,62881,0,0,2,2,0,0,0,0,0,...,0,0,0,0,0,1,0,0,3,0
3,64960,0,0,0,1,0,0,0,0,1,...,0,0,1,0,0,2,0,1,0,2
4,66179,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,3,0,0,1,3


## Committee table

### Importing Data

In [19]:
orgaos_membros = pd.read_sql("""
    SELECT
      idecadastro,
      idorgao,
      siglaorgao,
      nomeorgao,
      tipoorgao,
      nomepapel,
      datainicio,
      datafim
    FROM
      br_camdep_v2.orgaos_membros
    WHERE
      datainicio >= '2003-01-01';
""",con)
print('orgaos_membros OK')

orgaos_membros OK


In [20]:
orgaos_membros.idecadastro = np.nan_to_num(orgaos_membros.idecadastro).astype(int)
orgaos_membros = orgaos_membros[
    orgaos_membros.idecadastro.isin(dep_leg55.idecadastro)
]

In [21]:
orgaos_membros.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32446 entries, 14 to 53533
Data columns (total 8 columns):
idecadastro    32446 non-null int64
idorgao        32446 non-null int64
siglaorgao     32446 non-null object
nomeorgao      32446 non-null object
tipoorgao      32446 non-null object
nomepapel      32446 non-null object
datainicio     32446 non-null object
datafim        26093 non-null object
dtypes: int64(2), object(6)
memory usage: 2.2+ MB


## Plenary sessions votes table

### Importing Data

In [22]:
votacao_deputado = pd.read_sql("""
    SELECT DISTINCT
      idecadastro,
      codproposicao idproposicao,
      codsessao,
      datavotacao,
      voto,
      partido
    FROM
      c_camdep.camdep_proposicoes_votacao_deputado
    WHERE
      tipo SIMILAR TO '(PL|PLP|PLV|MPV|PDC|PEC)' AND
      datavotacao >= '2003-01-01'
""",con)
print('votacao_deputado OK')

votacao_bancada = pd.read_sql("""
    SELECT DISTINCT
      codproposicao,
      codsessao,
      datavotacao,
      sigla,
      orientacao
    FROM
      c_camdep.camdep_proposicoes_votacao_bancada
    WHERE
      tipo SIMILAR TO '(PL|PLP|PLV|MPV|PDC|PEC)' AND
      datavotacao >= '2003-01-01'
""",con)
print('votacao_bancada OK')

votacao_resumo = pd.read_sql("""
    SELECT DISTINCT
      codsessao,
      datavotacao,
      resumo,
      objvotacao
    FROM
      c_camdep.camdep_proposicoes_votacao
    WHERE
      tipo SIMILAR TO '(PL|PLP|PLV|MPV|PDC|PEC)' AND
      datavotacao >= '2003-01-01'
""",con)
print('votacao_resumo OK')

votacao_deputado OK
votacao_bancada OK
votacao_resumo OK


In [23]:
votacao_deputado.idecadastro = np.nan_to_num(votacao_deputado.idecadastro).astype(int)
votacao_deputado = votacao_deputado[
    votacao_deputado.idecadastro.isin(dep_leg55.idecadastro)
]

### Getting parties orientations

In [24]:
def preparate_orientations(sec,dt):
    
    orientations = votacao_bancada[
        (votacao_bancada.codsessao == sec) &
        (votacao_bancada.datavotacao == dt)
    ][['sigla','orientacao']].set_index('sigla').to_dict()['orientacao']

    dic = {}
    for (abbr,orientation) in orientations.items():
        abbr = abbr.replace('Repr.','').strip('.')
        if not abbr.isupper():
            abbrs = re.findall('[A-Z][^A-Z]*', abbr)
            if 'B' in abbrs:
                abbrs.remove('B')
                if 'Cdo' in abbrs:
                    abbrs.append('PCdoB')
                    abbrs.remove('Cdo')
                    abbrs.remove('P')
                elif 'Tdo' in abbrs:
                    abbrs.append('PTdoB')
                    abbrs.remove('Tdo')
                    abbrs.remove('P')
                elif 'Ptdo' in abbrs:
                    abbrs.append('PTdoB')
                    abbrs.remove('Ptdo')
            for i in abbrs:
                dic[i.upper()] = orientation
        else:
            dic[abbr] = orientation
    return dic
        
orientations_dic = {}
section_date = votacao_deputado[['codsessao','datavotacao']].drop_duplicates()

for sec,dt in tqdm(section_date.to_dict(orient='split')['data']):
    try:
        orientations_dic[sec][str(dt)] = preparate_orientations(sec,dt)
    except KeyError:
        orientations_dic[sec] = {str(dt): preparate_orientations(sec,dt)}

100%|██████████| 2088/2088 [00:16<00:00, 130.05it/s]


In [25]:
def get_orientation(row):
    try:
        orientation = orientations_dic[row.codsessao][str(row.datavotacao)][row.partido.upper()]
        return orientation
    except KeyError:
        return None
    
votacao_deputado.loc[:,'orientacaobancada'] = votacao_deputado.apply(get_orientation,axis=1)

### Merging the results

In [26]:
votacao_deputado = votacao_deputado.merge(
    votacao_resumo,
    left_on=['codsessao','datavotacao'],
    right_on=['codsessao','datavotacao'],
    how='left'
)

### Getting the propositions infos 

In [27]:
votacao_deputado = votacao_deputado.merge(
    proposicoes_id.loc[:,['idproposicao','nomeproposicao','ementa']],
    left_on='idproposicao',
    right_on='idproposicao',
    how='left'
)

In [28]:
votacao_deputado.head(2)

,idecadastro,idproposicao,codsessao,datavotacao,voto,partido,orientacaobancada,resumo,objvotacao,nomeproposicao,ementa
0,4930,14245,4872,2013-04-09 21:10:00,Sim,PTB,Obstrução,Prejudicado o requerimento por falta de quórum...,REQUERIMENTO DE RETIRADA DE PAUTA,PEC 3/1999,"Altera os arts. 27, 28, 29, 44 e 82 da Constit..."
1,4930,14271,3725,2009-05-20 20:26:00,Sim,PMN,Obstrução,Aprovado o Substitutivo adotado pela Comissão ...,SUBSTITUTIVO - PRIMEIRO TURNO,PEC 22/1999,Autoriza o divórcio após 1 (um) ano de separaç...


In [ ]:
# dep_leg55.to_sql('parlamentares',con,schema='viz_painel_parlamentares',if_exists='replace')
# parlamentares_tema.to_sql('parlamentares_temas',con,schema='viz_painel_parlamentares',if_exists='replace')
# proposicoes_id.to_sql('proposicoes',con,schema='viz_painel_parlamentares',if_exists='replace')
# orgaos_membros.to_sql('orgaos',con,schema='viz_painel_parlamentares',if_exists='replace')
votacao_deputado.to_sql('votacoes',con,schema='viz_painel_parlamentares',if_exists='replace')

In [ ]:
'a'

In [29]:
idecadastro = 73486
#idecadastro = 73584
data = requests.get('http://172.16.4.226:5001/temas/{}'.format(idecadastro)).json()
if 'data' not in data:
#     return None
    print('error')
data = data['data'][0]
data.pop('idecadastro')
data.pop('index')

sorted_data = sorted(data.items(), key=operator.itemgetter(1))

top5 = [i for i in sorted_data if i[1] != 0][-5:]
#top5.extend([(' ' * i, 0) for i in range(5 - len(top5))])

others = [i for i in sorted_data if i[1] != 0][:-5]
full = [('Outros',sum([i[1] for i in others]))] + top5
full = tuple(zip(*full))

themes, amount = full

data = go.Bar(
    orientation='h',
    y=themes,
    x=amount,
    #text=['{}<br>{} proposições'.format(i,j) for i,j in zip(themes,amount)],
    text=themes,
    textposition = 'outside',
    marker={'color': "#00B3E0"}
)

annotations = []
shapes = []
for i in range(len(amount)):
    v = amount[i]
    if v > 10: l = 5
    elif v > 2: l = 2
    else: l = 0.5
        
    if v > 200:
        text = '{} - {}'.format(themes[i],v)
        annotations.append(dict(
            y = i,
            x = 250 - len(text),
            text = text,
            showarrow = False,
            font = dict(
                color = "white",
                size = 12,
                family='Lato'
            ),
        ))
        annotations.append(dict(
            y = i+.55,
            x = 250 - len(text),
            text = 'CONTINUA ->    ',
            showarrow = False,
            font = dict(
                color = "rgb(170, 170, 170)",
                size = 12,
                family='Lato'
            ),
        ))
        shapes = [{
            'type': 'line',
            'x0': 250, 'y0': -.5,
            'x1': 250, 'y1': 6,
            'line': {
                'color': 'rgb(170, 170, 170)',
                'width': 3,
                'dash': 'dot'
            }
        }]
    else:
        annotations.append(dict(
            y = i,
            x = v-l,
            text = str(v),
            showarrow = False,
            font = dict(
                color = "white",
                size = 12,
                family='Lato'
            ),
        ))

layout = go.Layout(
    margin=go.Margin(
        l=20,
        t=10,
        pad=0
    ),
    xaxis=dict(
        ticks='',
        range=[0,250],
        tickfont=dict(
            size=10,
            family='Lato'
        ),
        zeroline=False,
        title='Quantidade'
    ),
    yaxis=dict(
        ticks='',
        showticklabels=False,
        title='Temas'
    ),
    annotations=annotations,
    shapes=shapes
)

fig = go.Figure(data=[data], layout=layout)
offline.iplot(fig)